In [1]:
import random
import pickle
import re

wordnet = {}
with open("./data/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)


# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\admin\AppData\Local\Temp\ipykernel_25284\3047796988.py:138: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [3]:
EDA('와플대학은 와플 중에 최고다!')

['와플대학은 와플 중에 최고다!',
 '와플 와플대학은 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '와플대학은 와플 중에 최고다!',
 '중에 와플 와플대학은 최고다!',
 '와플대학은 와플 중에 최고다!']

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('./data/부정_일상문.xlsx')
df

,0,emo_num,emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,0,당황
1,그냥 내 느낌일뿐겠지?,0,걱정
2,아직너무초기라서 그런거죠?,0,걱정
3,유치원버스 사고 낫다던데,0,걱정
4,근데 원래이런거맞나요,0,불안
...,...,...,...
6950,그러니일본놈들이한국사람을더우습게알고함부로대하는거아닌가?,0,불만
6951,위증은 무쟈껀 제자리 화형으로 다스려야 함.,0,분노
6952,짱개와는 연을 끊는게 상책이다,0,분노
6953,탄핵을 외친자들에게 물어보라!,0,NaN


In [3]:
df = df.dropna()

In [4]:
df['emotion'].unique()

array(['당황', '걱정', '불안', '궁금', '슬픔', '답답', '불만', '불편', '무서움', '짜증', '조롱',
       '보통', '허무함', '만족', '분노', '편안', '상처', '행복', '흥미', '감사'],
      dtype=object)

In [5]:
# emotion 값에 따라 값 부여
emo_ = []
for i in df['emotion']:
    emo_.append(i)
    
    
emo_num = []

for i in range(0,len(emo_)):
    if emo_[i] == '분노' :
        emo_num.append(6)
    elif emo_[i] == '슬픔':
        emo_num.append(7)
    elif emo_[i] == '당황':
        emo_num.append(8)
    elif emo_[i] == '상처':
        emo_num.append(9)
    elif emo_[i] == '불안':
        emo_num.append(10)
    elif emo_[i] == '짜증':
        emo_num.append(11)
    elif emo_[i] == '답답':
        emo_num.append(12)
    elif emo_[i] == '허무함':
        emo_num.append(13)
    elif emo_[i] == '조롱':
        emo_num.append(14)
    elif emo_[i] == '불편':
        emo_num.append(15)
    elif emo_[i] == '걱정':
        emo_num.append(16)
    elif emo_[i] == '불만':
        emo_num.append(17)
    elif emo_[i] == '무서움':
        emo_num.append(18)
    elif emo_[i] == '편안':
        emo_num.append(0)
    elif emo_[i] == '기쁨':
        emo_num.append(1)
    elif emo_[i] == '행복':
        emo_num.append(2)
    elif emo_[i] == '감사':
        emo_num.append(3)
    elif emo_[i] == '만족':
        emo_num.append(4)
    elif emo_[i] == '흥미':
        emo_num.append(5)
    elif emo_[i] == '보통':
        emo_num.append(19)
    else :
        emo_num.append(20)

In [6]:
df = df.reset_index(drop=True)

In [7]:
df['emo_num'] = pd.DataFrame(emo_num)
df

,0,emo_num,emotion
0,언니 동생으로 부르는게 맞는 일인가요..??,8,당황
1,그냥 내 느낌일뿐겠지?,16,걱정
2,아직너무초기라서 그런거죠?,16,걱정
3,유치원버스 사고 낫다던데,16,걱정
4,근데 원래이런거맞나요,10,불안
...,...,...,...
6818,새해는 직무정지되서 꼴깝떠는거 안볼 줄알았더만 18연초부터 드럽게 재수없네,11,짜증
6819,그러니일본놈들이한국사람을더우습게알고함부로대하는거아닌가?,17,불만
6820,위증은 무쟈껀 제자리 화형으로 다스려야 함.,6,분노
6821,짱개와는 연을 끊는게 상책이다,6,분노


In [8]:
df.to_excel('./data/부정일상문.xlsx', index=False)

In [10]:
df_h = df[df['emotion']=='허무함']
df_h = df_h.reset_index(drop=True)
df_h

,0,emo_num,emotion
0,무어라 할 말이 없네....,13,허무함
1,진짜 허탈하네요,13,허무함
2,아니야정말 그런의미일수도있어 또기대하게되고..,13,허무함
3,지금 보니 우연인지 뭔지...,13,허무함
4,이렇게 변명하는 제 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;,13,허무함
5,11일 2층 10구역 가는데..누나는 똥손이란 걸 다시금 깨달았어요..,13,허무함
6,이란갈껄 담에 이란승에 몰빵,13,허무함
7,인생지사 허무맹랑 하네.,13,허무함
8,그냥 허망하다는 말밖에 안나옵니다..,13,허무함
9,그런데 아무 답장이 없더라구요....,13,허무함


In [14]:
len(df_h)

25

In [11]:
 df_h[0][0]

' 무어라 할 말이 없네....'

In [40]:
eda =[]
for i in df_h[0]:
    # print(i)
    a = EDA(i)
    b = list(set(a))
    eda.append(b)

In [41]:
eda

[['무어라 할 말이 없네....', '말이 할 무어라 없네....', '없네.... 할 말이 무어라', ' 무어라 할 말이 없네....'],
 ['허탈하네요 진짜', '진짜 허탈하네요', '진 허탈하네요', '진짜 진 허탈하네요', '진 진짜 허탈하네요'],
 ['그런의미일수도있어 아니야정말 또기대하게되고..',
  '아니야정말 그런의미일수도있어  또기대하게되고..',
  '아니야정말 그런의미일수도있어 또기대하게되고..'],
 [' 지금 보니 우연인지 뭔지...',
  '지금 우연인지 보니 뭔지...',
  '우연인지 보니 지금 뭔지...',
  '지금 보니 우연인지 뭔지...'],
 ['이렇게 변명하는 제 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 변명하는 제 자신이 제 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 변명하는 생 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 변명하는 제 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 변명하는 남 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 변명하는 제 왜 자신이 이렇게 웃겨보일까요..-ㅁ-?;;',
  '이렇게 왜 제 자신이 변명하는 이렇게 웃겨보일까요..-ㅁ-?;;',
  ' 이렇게 변명하는 제 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;'],
 ['11일 2층 10구역 가는데..누나는 똥손이란 걸 다시금 깨달았어요..',
  '11일 10구역 2층 가는데..누나는 똥손이란 걸 다시금 깨달았어요..',
  '11일 10구역 똥손이란 걸 다시금 깨달았어요..',
  '11일 2층 10구역 가는데..누나는 걸 깨달았어요..',
  '11일 2층 10구역 가는데..누나는 똥손이란 다시금 깨달았어요..'],
 ['이란승에 담에 이란갈껄 몰빵', '이란갈껄 담에 이란승에 몰빵', '이란갈껄 이란승에 담에 몰빵', '이란갈껄 담에 몰빵'],
 ['허무맹랑 인생지사 하네.', '인생지사 허무맹랑 하네.'],
 ['그냥 말밖에 안나옵니다..', '그냥 말밖에

In [56]:
augm = sum(eda, [])
augm

['무어라 할 말이 없네....',
 '말이 할 무어라 없네....',
 '없네.... 할 말이 무어라',
 ' 무어라 할 말이 없네....',
 '허탈하네요 진짜',
 '진짜 허탈하네요',
 '진 허탈하네요',
 '진짜 진 허탈하네요',
 '진 진짜 허탈하네요',
 '그런의미일수도있어 아니야정말 또기대하게되고..',
 '아니야정말 그런의미일수도있어  또기대하게되고..',
 '아니야정말 그런의미일수도있어 또기대하게되고..',
 ' 지금 보니 우연인지 뭔지...',
 '지금 우연인지 보니 뭔지...',
 '우연인지 보니 지금 뭔지...',
 '지금 보니 우연인지 뭔지...',
 '이렇게 변명하는 제 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 변명하는 제 자신이 제 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 변명하는 생 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 변명하는 제 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 변명하는 남 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 변명하는 제 왜 자신이 이렇게 웃겨보일까요..-ㅁ-?;;',
 '이렇게 왜 제 자신이 변명하는 이렇게 웃겨보일까요..-ㅁ-?;;',
 ' 이렇게 변명하는 제 자신이 왜 이렇게 웃겨보일까요..-ㅁ-?;;',
 '11일 2층 10구역 가는데..누나는 똥손이란 걸 다시금 깨달았어요..',
 '11일 10구역 2층 가는데..누나는 똥손이란 걸 다시금 깨달았어요..',
 '11일 10구역 똥손이란 걸 다시금 깨달았어요..',
 '11일 2층 10구역 가는데..누나는 걸 깨달았어요..',
 '11일 2층 10구역 가는데..누나는 똥손이란 다시금 깨달았어요..',
 '이란승에 담에 이란갈껄 몰빵',
 '이란갈껄 담에 이란승에 몰빵',
 '이란갈껄 이란승에 담에 몰빵',
 '이란갈껄 담에 몰빵',
 '허무맹랑 인생지사 하네.',
 '인생지사 허무맹랑 하네.',
 '그냥 말밖에 안나옵니다..',
 '그냥 말밖에 허망하다는 안나옵니다..',
 '그냥

In [57]:
DF = pd.DataFrame(augm)

In [59]:
DF.to_excel('./data/허무함.xlsx', index=False)

### 상처 증강

In [61]:
df_s = df[df['emotion']=='상처']
df_s = df_s.reset_index(drop=True)
df_s

,0,emo_num,emotion
0,머랄까 가끔 정적이 흘러요..,9,상처
1,사진첩을 보게 되었는데 전남친 사진이 있더라구요,9,상처
2,저말고 다른여친이 있떠군요..,9,상처
3,외로워서...너무할일이없어서...취정게에 뻘낚시글을 썼다가 글삭에 경고크리먹었음.,9,상처
4,금세 사랑이 식었네...우째...우리가 그정도밖에 안 됐어?,9,상처
5,근데 남편는 아이한테배신감땜에 상처받았으가봅니다,9,상처
6,관계후 다음날에갑자기 생각할 시간을 갖자고통보받았습니다..,9,상처
7,별걸 아니게 들릴 수 있겠지만 저힌테는 큰 아픔이자 억울함입니다.,9,상처
8,제가 이상한거죠?,9,상처
9,뭐하며 맘을 다스려야할까,9,상처


In [62]:
eda =[]
for i in df_s[0]:
    # print(i)
    a = EDA(i)
    b = list(set(a))
    eda.append(b)

In [63]:
augm = sum(eda, [])
DF = pd.DataFrame(augm)


In [65]:
DF.to_excel('./data/상처.xlsx', index=False)

### 당황 증강

In [66]:
df_d = df[df['emotion']=='당황']
df_d = df_d.reset_index(drop=True)

eda =[]
for i in df_d[0]:
    # print(i)
    a = EDA(i)
    b = list(set(a))
    eda.append(b)
    
augm = sum(eda, [])
DF = pd.DataFrame(augm)
DF

,0
0,언니 동생으로 부르는게 맞는 일인가요..??
1,언니 동생으로 언 부르는게 맞는 일인가요..??
2,언니 동생으로 부르는게 언 맞는 일인가요..??
3,언니 동생으로 부르는게 일인가요..?? 맞는
4,언 언니 동생으로 부르는게 맞는 일인가요..??
...,...
862,이짓할줄예상은했는데 설마 근데진짜한다
863,설마 이짓할줄예상은했는데 근데진짜한다
864,근데진짜한다 설마 이짓할줄예상은했는데
865,친일파일줄이야... 티파니가


In [ ]:
DF.to_excel('./data/당황.xlsx', index=False)


#### augmentation 함수로 정리

In [71]:
def AUG(emo):
    df_d = df[df['emotion']==emo]
    df_d = df_d.reset_index(drop=True)

    eda =[]
    for i in df_d[0]:
        # print(i)
        a = EDA(i)
        b = list(set(a))
        eda.append(b)
        
    augm = sum(eda, [])
    return augm


In [72]:
AUG('답답')

['아~ 어떻게 정말 해야되죠..1',
 '아~ 정말 어떻게 해야되죠..1',
 '해야되죠..1 정말 어떻게 아~',
 '아~ 정말 어떻게',
 '방법을 모르겠네요... 잘',
 '잘 방법을 모르겠네요...',
 '방법을 잘 모르겠네요...',
 '뭘해야할지 모르겠네요',
 '모르겠네요 뭘해야할지',
 '근데 정보가 많진 않으니 선뜻 지원하기가 쉽지않네요',
 '근데 정보가 선뜻 않으니 많진 지원하기가 쉽지않네요',
 '정보가 많진 지원하기가 쉽지않네요',
 '선뜻 정보가 많진 않으니 근데 지원하기가 쉽지않네요',
 '정보가 근데 많진 않으니 선뜻 지원하기가 쉽지않네요',
 '정보가 많진 않으니 지원하기가 쉽지않네요',
 '진짜 지면 중국한테도 개망신이야...',
 '중국한테도 진 지면 진짜 개망신이야...',
 '중국한테도 지면 진 개망신이야...',
 '진 중국한테도 지면 진짜 개망신이야...',
 '중국한테도 진짜 지면 개망신이야...',
 '중국한테도 지면 진짜 개망신이야...',
 '정말 다 같은 인간은 아닌듯',
 '아닌듯 인간이라구 다 같은 인간은 정말',
 '정말 인간이라구 다 같은 인간은 아닌듯',
 '정말 같은 다 인간이라구 인간은 아닌듯',
 '정말 인간이라구 인간은 같은 다 아닌듯',
 '난해하다......',
 '진짜..어이없네',
 '원톱가면......제발......이것만은... 흥민이는',
 '흥민이는 원톱가면......제발......이것만은...',
 '흥민이는',
 '물론 시즌말에 가서 떨어질꺼라 예상은 했다만,스크는 나름대로 잘 나가다가 왜 성적이 곤두박질 칠겨?',
 '물론 시즌말에 가서 떨어질꺼라 곤두박질 했다만,스크는 나름대로 잘 나가다가 왜 성적이 예상은 칠겨?',
 '물론 시즌말에 가서 떨어질꺼라 예상은 나가다가 나름대로 잘 했다만,스크는 왜 성적이 곤두박질 칠겨?',
 '물론 시즌말에 떨어질꺼라 했다만,스크는 나름대로 잘 나가다가 왜 성적이 곤두박질 칠겨?',
 '참 이노인네는',
 '이노인네는',
 '이노인네는 참'